In [7]:
from datasets import load_dataset

ds = load_dataset("keremberke/pokemon-classification", name="full")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset pokemon-classification downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/keremberke___pokemon-classification/full/1.0.0/5193d0f863ed08164636296506df483caa6c9965bc05198ec2136417398e4b28. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
import torch

In [4]:
from pathlib import Path
Path.cwd()

PosixPath('/home/ubuntu/transformer_framework/config')

In [10]:
ds_train = ds['train']

In [12]:
ds_validation = ds['validation']

In [13]:
ds_train

Dataset({
    features: ['image_file_path', 'image', 'labels'],
    num_rows: 4869
})

In [15]:
import torchvision.transforms as tv 
imgnet_normalize = tv.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )

In [16]:

train_transforms = tv.Compose(
            [
                tv.Resize((256, 256)),
                tv.RandomResizedCrop(224),
                tv.RandomHorizontalFlip(),
                tv.ToTensor(),
                imgnet_normalize,
            ]
        )

val_transforms = tv.Compose(
            [
                tv.Resize((256, 256)),
                tv.CenterCrop(224),
                tv.ToTensor(),
                imgnet_normalize,
            ]
        )


In [17]:
import torchvision.transforms as tv

# ----- core transforms ----------

_imgnet_normalize = tv.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

train_transforms = tv.Compose(
        [
            tv.Resize((256, 256)),
            tv.RandomResizedCrop(224),
            tv.RandomHorizontalFlip(),
            tv.ToTensor(),
            _imgnet_normalize,
        ]
    )

validation_transforms = tv.Compose(
    [
        tv.Resize((256, 256)),
        tv.CenterCrop(224),
        tv.ToTensor(),
        _imgnet_normalize,
    ]
)

# ---- functions ---------

def train_batch_transforms(samples):
    samples["pixel_values"] = [train_transforms(image.convert("RGB")) for image in samples["image"]]
    del samples["image"]
    return samples

def validation_batch_transforms(samples):
    samples["pixel_values"] = [validation_transforms(image.convert("RGB")) for image in samples["image"]]
    del samples["image"]
    return samples



def get_datasets():
    """cover function for handling loading the training and validation dataset"""
    """dataset loading"""
    from datasets import load_dataset

    ds = load_dataset("keremberke/pokemon-classification", name="full")

    ds_train = ds["train"]
    ds_validation = ds["validation"]

    
    train_dataset = ds_train.with_transform(train_batch_transforms)
    validation_dataset = ds_validation.with_transform(validation_batch_transforms)

    return train_dataset, validation_dataset

In [18]:
dst, dsv = get_datasets()

Found cached dataset pokemon-classification (/home/ubuntu/.cache/huggingface/datasets/keremberke___pokemon-classification/full/1.0.0/5193d0f863ed08164636296506df483caa6c9965bc05198ec2136417398e4b28)


  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
dst

Dataset({
    features: ['image_file_path', 'image', 'labels'],
    num_rows: 4869
})

In [25]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dsv, batch_size=4)


In [26]:
for batch in dataloader:
    x = batch['pixel_values']
    y = batch['labels']
    break


In [27]:
x.shape

torch.Size([4, 3, 224, 224])

In [28]:
y.shape

torch.Size([4])

In [29]:
y

tensor([127, 127, 127, 127])